# Dogs vs cats neural network

You can find the kaggle competition information here: https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition

worked with Mkrtich Ohanyan: https://www.mkrtich.com/

In [20]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [21]:
import numpy as np
import pandas as pd
import os
import zipfile

# extract files
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip","r") as z:
    z.extractall(".")

with zipfile.ZipFile("/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip","r") as z:
    z.extractall(".")

In [22]:
import cv2
import matplotlib.pyplot as plt

#main directory=/kaggle/working/   
main_dir="/kaggle/working/"    

#train path and test path 
train_dir = "train"
test_dir="test"

#train and test paths
train_path=os.path.join(main_dir,train_dir)
test_path=os.path.join(main_dir,test_dir)


#Look at a cat/dog in color.
for p in os.listdir(train_path):
    category = p.split(".")[0]
    img_array = cv2.imread(os.path.join(train_path,p),cv2.IMREAD_COLOR)
    RGB_img = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    new_img_array = cv2.resize(RGB_img, dsize=(80, 80))
    plt.imshow(new_img_array)
    break

Create X and y.

In [23]:
X = []
y = []
test=[]
convert = lambda category : int(category == 'dog')
def create_test_data(path):
    for p in os.listdir(path):
        category = p.split(".")[0]
        category = convert(category)
        img_array = cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)
        #RGB_img = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        new_img_array = cv2.resize(img_array, dsize=(80, 80))
        X.append(new_img_array)
        y.append(category)
        
def create_test1_data(path):
    for p in os.listdir(path):
        category = p.split(".")[0]
        category = convert(category)
        img_array = cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)
        #RGB_img = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        new_img_array = cv2.resize(img_array, dsize=(80, 80))
        test.append(new_img_array)


In [24]:
create_test_data(train_path)
create_test1_data(test_path)
X = np.array(X)
y = np.array(y)
test=np.array(test)

In [25]:
X.shape,y.shape, test.shape

Train-test-split

In [26]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)

Reshape X_train, X_test, y_train,y_test.

In [27]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [28]:
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))
X_train = np.array(X_train).reshape(-1, 80,80,1)
X_test=np.array(X_test).reshape(-1,80,80,1)

In [29]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [30]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D


model = keras.Sequential([
    #input layer
    layers.InputLayer(input_shape=(80,80,1)),
    
    ###Convolution Block 1###
    #Conv2D looks for features
    layers.Conv2D(filters=32,kernel_size=3,activation='relu'),
    #MaxPooling2D helps reduce overfitting
    layers.MaxPooling2D(strides=2, padding='same'),
    #Dropout helps reduce overfitting
    layers.Dropout(rate=0.3),
    #BatchNormalization standardizes
    #layers.BatchNormalization(),
    
    
    ###Convolution Block 2###
    #Conv2D looks for features
    layers.Conv2D(filters=64,kernel_size=3,activation='relu'),
    #MaxPooling2D helps reduce overfitting
    layers.MaxPooling2D(strides=2, padding='same'),
    #Dropout helps reduce overfitting
    layers.Dropout(rate=0.3),
    #BatchNormalization standardizes
    #layers.BatchNormalization(),
    
    
    ###Convolution Block 3###
    #Conv2D looks for features
    layers.Conv2D(filters=128,kernel_size=3,activation='relu'),
    #MaxPooling2D helps reduce overfitting
    layers.MaxPooling2D(strides=2, padding='same'),
    #Dropout helps reduce overfitting
    layers.Dropout(rate=0.3),
    #BatchNormalization standardizes
    #layers.BatchNormalization(),
    
    #fully connected layers
    #puts dimensions together
    layers.Flatten(),
    #helps reduce overfitting
    layers.Dropout(rate=0.3),
    #output layer
    layers.Dense(1,activation='sigmoid'),
])

In [31]:
model.summary()

Input layer

Conv2D layer-- looks for "features" or edges

ReLu layer

Pooling layer

In [38]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [39]:
batch_size = 16
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])



## this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow(X_train, y_train, batch_size = 32)

# this is a similar generator, for validation data
validation_generator = test_datagen.flow(X_test,y_test,batch_size=32)



model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save_weights('first_try.h5')  # always save your weights after training or during training

# early_stopping = keras.callbacks.EarlyStopping(
#     patience=5,
#     min_delta=0.01,
#     restore_best_weights=True,
# )

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
#     callbacks=[early_stopping],
    batch_size=32,
    epochs=150,
)


In [40]:
history_df = pd.DataFrame(history.history)
history_df.to_csv('history.csv')

In [41]:
#test= test.values.astype('float32')
test = test.reshape(test.shape[0], 80, 80,1)
label=model.predict(test)

In [42]:
label1=[]

i=0


for i in range(0,len(label)):
    label1.append(label[i][0])
    i=i+1
    
print(len(label1),len(label))


In [43]:
predictions=np.clip(label1,0.05,0.95)

In [44]:
len(predictions)

In [45]:
my_submission = pd.DataFrame({'id': list(range(1,len(label)+1)), 'label': predictions})
my_submission.to_csv('submission.csv', index=False)